# Data-prepareation

## use numpy read file + use torch read data

In [3]:
file = "./data/arch/Train/7_TR_cloister.txt"
time1 = datetime.datetime.now()
data = np.loadtxt(file)
scene_points = torch.from_numpy(data[:, 0:3].astype('float32'))
segment_label = torch.from_numpy(data[:,6].astype('int32'))
time2 = datetime.datetime.now()
print(scene_points.shape)
print(segment_label.shape)
print(time2-time1)

torch.Size([15740229, 3])
torch.Size([15740229])
0:03:52.921357


## gen batch to hdf5 file

### pointnet method

In [1]:
def sample_data(data, num_sample):
    """ data is in N x ...
        we want to keep num_samplexC of them.
        if N > num_sample, we will randomly keep num_sample of them.
        if N < num_sample, we will randomly duplicate samples.
    """
    N = data.shape[0]
    if (N == num_sample):
        return data, range(N)
    elif (N > num_sample):
        sample = np.random.choice(N, num_sample)
        return data[sample, ...], sample
    else:
        sample = np.random.choice(N, num_sample-N)
        dup_data = data[sample, ...]
        return np.concatenate([data, dup_data], 0), list(range(N))+list(sample)

In [2]:
def sample_data_label(data, label, num_sample):
    new_data, sample_indices = sample_data(data, num_sample)
    new_label = label[sample_indices]
    return new_data, new_label

In [5]:
import numpy as np
NUM_POINT = 4096
H5_BATCH_SIZE = 1000
data_dim = [NUM_POINT, 3]
label_dim = [NUM_POINT]

batch_data_dim = [H5_BATCH_SIZE] + data_dim
batch_label_dim = [H5_BATCH_SIZE] + label_dim
h5_batch_data = np.zeros(batch_data_dim, dtype = np.float32)
h5_batch_label = np.zeros(batch_label_dim, dtype = np.uint8)
buffer_size = 0  # state: record how many samples are currently in buffer
h5_index = 0 # state: the next h5 file to save

print([H5_BATCH_SIZE])
print(batch_data_dim)
print(h5_batch_data.shape)
print(h5_batch_label.shape)

[1000]
[1000, 4096, 3]
(1000, 4096, 3)
(1000, 4096)


In [54]:
data_size = current_data.shape[0]
h5_batch_data[buffer_size:buffer_size+data_size, ...] = current_data
buffer_size += data_size

print(h5_batch_data.shape)

(1000, 2048, 3)


## Generate hdf5 format (1000, 2048, 10) point clouds

In [44]:
#generate h5 files for scene 7
!python pre_hdf5/gen_arch_h5_pointnet.py --split_file '7_SMV_chapel_24.txt'

train/7_SMV_chapel_24.txt
input file: /home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud/data/arch/train/7_SMV_chapel_24.txt
input scene: /home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud/data/arch/train/7_SMV_chapel_24.txt and shape: (3571064, 7)
block number: 352, 8425
^C
Traceback (most recent call last):
  File "pre_hdf5/gen_arch_h5_pointnet.py", line 122, in <module>
    data, labels = common.scene2blocks_wrapper(filepath,NUM_POINT, block_size=args.block_size, stride=args.stride, random_sample=False, sample_num=None)
  File "/home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud/pre_hdf5/common.py", line 164, in scene2blocks_wrapper
    random_sample, sample_num, sample_aug)
  File "/home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud/pre_hdf5/common.py", line 155, in scene2blocks_plus
    random_sample, sample_num, sample_aug)
  File "/home/yw/Documents/experiment/Un

In [ ]:
#generate h5 files for test data
!python pre_hdf5/gen_arch_h5_pointnet.py --split 'test'

## test PointCNN point clouds split algorithm

In [50]:
!python pre_hdf5/gen_arch_h5_pointcnn.py

Namespace(block_size=1.5, folder=None, grid_size=0.03, max_point_num=8192, random_sample=False, save_ply=False)
2020-11-24 22:02:09.794724-Loading /home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud/data/arch/test/A_SMG_portico.txt...
2020-11-24 22:06:16.629737-Computing block id of 17798012 points...
2020-11-24 22:06:19.403253-Collecting points belong to each block...
2020-11-24 22:06:47.949717-A_SMG_portico.txt is split into 432 blocks.
2020-11-24 22:06:47.956153-32 of 432 blocks are merged.
2020-11-24 22:07:53.162339-Saving /home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud/data/arch/test/A_SMG_portico.txt_8196_zero_0.h5...
2020-11-24 22:07:58.871601-Saving /home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud/data/arch/test/A_SMG_portico.txt_8196_zero_1.h5...
2020-11-24 22:08:02.120558-Computing block id of 17798012 points...
2020-11-24 22:08:04.733596-Collecting points belong to each block...
202

### test output shape

In [10]:
def load_h5(h5_filename):
    f = h5py.File(h5_filename, 'r')
    data = f['data'][:]
    label = f['label'][:]
    data_num = f['data_num'][...].astype(np.int32)
    labels_seg = f['label_seg'][...].astype(np.int64)
    if 'indices_split_to_full' in f:
        indices_split_to_full = f['indices_split_to_full'][...].astype(np.int64)
    return (data, label, data_num, labels_seg, indices_split_to_full)

In [11]:
import h5py
data_train, _, data_num_train, label_train, _ = load_h5("./data/arch/Train1/12_KAS_pavillion_1.txt_8192_half_0.h5")
print(data_train.shape)
print(label_train.shape)
print(data_num_train.shape)

(96, 8196, 6)
(96, 8196)
(96,)


### test load data function

In [20]:
def load_seg(filelist):
    points = []
    labels = []
    point_nums = []
    labels_seg = []
    indices_split_to_full = []

    for fn in filelist:
        data = h5py.File(fn, 'r')
        points.append(data['data'][...].astype(np.float32))
        labels.append(data['label'][...].astype(np.int64))
        point_nums.append(data['data_num'][...].astype(np.int32))
        labels_seg.append(data['label_seg'][...].astype(np.int64))
        if 'indices_split_to_full' in data:
            indices_split_to_full.append(data['indices_split_to_full'][...].astype(np.int64))

    return (np.concatenate(points, axis=0),
            np.concatenate(labels, axis=0),
            np.concatenate(point_nums, axis=0),
            np.concatenate(labels_seg, axis=0),
            np.concatenate(indices_split_to_full, axis=0) if indices_split_to_full else None)

In [16]:
# Prepare inputs
import os.path
import sys
from datetime import datetime
sys.path.append(os.path.join(".", 'utils'))
import pc_utils


filelist = "./data/arch/train_data_files.txt"
batch_size = 12
num_epochs = 256
print('{}-Preparing datasets...'.format(datetime.now()))
is_list_of_h5_list = not pc_utils.is_h5_list(filelist)
if is_list_of_h5_list:
    seg_list = pc_utils.load_seg_list(filelist)
    seg_list_idx = 0
    filelist_train = seg_list[seg_list_idx]
    seg_list_idx = seg_list_idx + 1
else:
    filelist_train =  filelist
print(filelist_train)
data_train, _, data_num_train, label_train, _ = pc_utils.load_seg(filelist_train)
print(data_train.shape)
print(data_num_train.shape)
print(label_train.shape)
num_train = data_train.shape[0]
point_num = data_train.shape[1]

print('{}-{:d} training samples.'.format(datetime.now(), num_train))
batch_num = (num_train * num_epochs + batch_size - 1) // batch_size
print('{}-{:d} training batches.'.format(datetime.now(), batch_num))

2020-10-29 17:36:45.085651-Preparing datasets...
./data/arch/./filelists/train_files_g_0.txt
(8192, 2048, 6)
(8192,)
(8192, 2048)
2020-10-29 17:36:45.708164-8192 training samples.
2020-10-29 17:36:45.708325-174763 training batches.


In [4]:
filelist = "data/arch/train_data_files_1.txt"
path_h5py_all = [line.strip()for line in open(filelist)]

In [ ]:
!python ./datasets/dataloader.py

In [8]:
!python main.py

Namespace(batch_size=16, dataset='shapenetcorev2', dropout=0.5, encoder='foldnet', epochs=10, eval=False, experiment_name=None, feat_dims=512, gpu_mode=False, k=None, model_path='', num_points=2048, num_workers=0, snapshot_interval=10, split='train', use_jitter=False, use_rotate=False, use_translate=False, workers=16)
-Preparing dataset...
-Loading ShapeNetCore dataset...
(2048, 2048, 3)
training set size:  100
training start!!!
total training nuber: 100total batch number: 6 .
batch idx: 0/6 in 0/10 epoch...
batch idx: 1/6 in 0/10 epoch...
batch idx: 2/6 in 0/10 epoch...
batch idx: 3/6 in 0/10 epoch...
batch idx: 4/6 in 0/10 epoch...
batch idx: 5/6 in 0/10 epoch...
batch idx: 6/6 in 0/10 epoch...
Epoch 1: Loss 10304.005859375, time 34.5129s
Save model to snapshort/Reconstruct1106233149/models/shapenetcorev2_1.pkl
Save model to snapshort/Reconstruct1106233149/models/shapenetcorev2_best.pkl
end epoch 0, training loss: 10304.006
total training nuber: 100total batch number: 6 .
batch idx: 

In [23]:
!python main.py --eval --model_path "./snapshort/Reconstruct1106233149/models/shapenetcorev2_best.pkl"

Namespace(batch_size=16, dataset='shapenetcorev2', dropout=0.5, encoder='foldnet', epochs=10, eval=True, experiment_name=None, feat_dims=512, gpu_mode=False, k=None, model_path='./snapshort/Reconstruct1106233149/models/shapenetcorev2_best.pkl', num_points=2048, num_workers=0, snapshot_interval=10, split='train', use_jitter=False, use_rotate=False, use_translate=False, workers=16)
-Preparing evaluation dataset...
-Preparing ShapeNetCore evaluation dataset...
(2048, 2048, 3)
training set size: 100
(2048, 2048, 3)
testing set size: 100
Load model from ./snapshort/Reconstruct1106233149/models/shapenetcorev2_best.pkl.
size of generate traing set: (100, 512) ,(100,)
Train set 0 for SVM saved.
finish generating train set for SVM.
batch idx: 0 for generating test set for SVM...
batch idx: 1 for generating test set for SVM...
batch idx: 2 for generating test set for SVM...
batch idx: 3 for generating test set for SVM...
batch idx: 4 for generating test set for SVM...
batch idx: 5 for generating

In [1]:
!tensorboard --logdir tensorboard --bind_all

TensorFlow installation not found - running with reduced feature set.
TensorBoard 2.3.0 at http://yw:6006/ (Press CTRL+C to quit)
^C


In [12]:
!python visualization.py --item 1 --model_path=snapshot/Reconstruct1106233149/models --draw_original --draw_source_points

Namespace(dataset='shapenetcorev2', dataset_root='./data', draw_original=True, draw_source_points=True, encoder='foldingnet', exp_name=None, feat_dims=512, item=1, k=None, model_path='snapshot/Reconstruct1106233149/models', num_points=2048, split='train')
Dataset: shapenetcorev2, split: train, item: 1, category: car


In [4]:
!source /home/yw/Downloads/mitsuba2/setpath.sh

In [7]:
!mitsuba "mitsuba/Reconstruct1106233149/shapenetcorev2/train1/shapenetcorev2_train1_car_epoch0.xml"

/bin/bash: mitsuba: command not found
